<a href="https://colab.research.google.com/github/antoniorendonc/DataSciencePortfolio/blob/main/ETL_Excel_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import auth
from google.cloud import bigquery

# Inicio del proceso ETL para cargar un archivo de Excel a BigQuery
# Antonio Rendon
# LinkedIn: https://www.linkedin.com/in/antoniorendon/
# Video: (URL del video si está disponible)

# EXTRACT
# Cargar el dataset desde un archivo de Excel, omitiendo las primeras 4 filas
df = pd.read_excel('/content/Adidas US Sales Datasets.xlsx', skiprows=4)

# Revisar las dimensiones iniciales del DataFrame para verificar la carga correcta
print("Dimensiones iniciales del DataFrame:", df.shape)
print("Primeras filas del DataFrame:", df.head())

# TRANSFORM
# Eliminar columnas innecesarias que no aportan a nuestro análisis
df.drop(['Unnamed: 0', 'Retailer ID'], axis=1, inplace=True)

# Renombrar columnas para hacer los nombres más claros y comprensibles
df.rename(columns={
    'Invoice Date': 'InvDate',
    'Price per Unit': 'PPU',
    'Units Sold': 'USold',
    'Total Sales': 'TotalSales',
    'Operating Profit': 'OpProfit',
    'Operating Margin': 'Margin',
    'Sales Method': 'Chanel'
}, inplace=True)

# Convertir la columna de fecha de factura a formato de fecha simple
df['InvDate'] = df['InvDate'].dt.date

# Ajustar los valores de TotalSales y OpProfit dividiéndolos por 10 para normalización
df['TotalSales'] = df['TotalSales'] / 10
df['OpProfit'] = df['OpProfit'] / 10

# LOAD
# Autenticación de usuario para Google Cloud, necesario para interactuar con BigQuery
auth.authenticate_user()
print('Usuario autenticado')

# Configuración del cliente de BigQuery con el proyecto específico
client = bigquery.Client(project='adidas-project-415820')

# Configuración para la carga de datos en BigQuery, con la opción de sobrescribir datos existentes
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

# Cargar los datos transformados en BigQuery, especificando la tabla de destino
table_id = 'adidas_ds.sales_2020_2021'
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()  # Esperar a que la carga se complete

# Verificar el estado del trabajo de carga para confirmar la finalización exitosa
if job.state == 'DONE' and job.error_result is None:
    print("La carga se completó exitosamente.")
else:
    print("El trabajo no se completó exitosamente. Errores:")
    for error in job.errors:
        print(error)

# VALIDATION
# Comparar el número de filas en el DataFrame original y en la tabla de BigQuery para validar la carga
table = client.get_table(table_id)  # Obtener detalles de la tabla en BigQuery
num_filas_df = df.shape[0]

if table.num_rows == num_filas_df:
    print(f"La carga fue exitosa. El DataFrame y la tabla en BigQuery tienen {num_filas_df} filas.")
else:
    print(f"Posible discrepancia en la carga. El DataFrame tiene {num_filas_df} filas, pero la tabla en BigQuery tiene {table.num_rows} filas.")
